In [1]:
%reset
%load_ext autoreload
%autoreload 2

import sys

sys.path.append("../")
import torch
import numpy as np
import defmod as dm
import multimodule_usefulfunctions as mm 
import matplotlib.pyplot as plt
import defmod.hamiltonian_multishape as hamiltonian
import model_graddesc as model


torch.set_default_tensor_type(torch.DoubleTensor)

Once deleted, variables cannot be recovered. Proceed (y/[n])? y


In [2]:
nb_pts1 = 10
nb_pts2 = 7
dim = 2

o1 = torch.tensor([0,0]).float()
o2 = torch.tensor([10,0]).float()
origin = [o1,o2]
r1 = 2
r2 = 1
radius = [r1, r2]
nb_pts = [nb_pts1, nb_pts2]
source = mm.multipleCircles(origin, radius, nb_pts)

origin_target = [[0,5], [10,-5]]
target = mm.multipleCircles(origin_target, radius, nb_pts)

In [3]:
# Defining the deformation modules
sigma1 = 10
manifold1 = dm.manifold.Landmarks(dim, nb_pts1, gd=source[0].view(-1))
trans1 = dm.deformationmodules.Translations(manifold1, sigma1)

sigma2 = 10
manifold2 = dm.manifold.Landmarks(dim, nb_pts2, gd=source[1].view(-1))
trans2 = dm.deformationmodules.Translations(manifold2, sigma2)

module_list = [trans1, trans2]
sigma_bg = 0.1
modules = dm.multishape.MultiShapeModule(module_list, sigma_bg)

In [4]:
constr_Id = dm.constraints.Identity()
H = hamiltonian.Hamiltonian_multi(modules, constr_Id)

In [5]:
gd0_list = [*source, source]
mom0_list = [*[torch.rand(gd.shape) for gd in modules.manifold.gd[:-1]], [torch.rand(gd.shape) for gd in modules.manifold.gd[-1]]]

# set parameters
reg_param = 10**-2 # regularisation parameter

#gd0_tensor = torch.cat([gd0_list[0], gd0_list[1], gd0_list[2][0], gd0_list[2][1]],0).requires_grad_().view(-1).double()
#mom0_tensor = torch.cat([mom0_list[0], mom0_list[1], mom0_list[2][0], mom0_list[2][1]],0).requires_grad_().view(-1).double()
    
EnergyFunctional = model.EnergyFunctional(modules, H, constr_Id, target, dim, reg_param)
gd0_tensor = EnergyFunctional.list2tensor(gd0_list)
mom0_tensor = EnergyFunctional.list2tensor(mom0_list)

#gradE = torch.autograd.grad(EnergyFunctional.energy_tensor(gd0_tensor, mom0_tensor), mom0_tensor, create_graph=True, allow_unused=True)

In [6]:
def test_gradcheck_tensor2list():
    def tensor2list(x):
 
        return EnergyFunctional.tensor2list(x)[2]
    
    return torch.autograd.gradcheck(tensor2list, gd0_tensor, raise_exception=True)

test_gradcheck_tensor2list()

True

In [7]:
def test_gradcheck_attach():
    def attach(gd, t):
        t = [t[:nb_pts1,:], t[nb_pts1:,:]]
        gd = EnergyFunctional.tensor2list(gd)
        EnergyFunctional.modules.manifold.fill_gd(gd)
        return EnergyFunctional.attach(t)
    
    return torch.autograd.gradcheck(attach, (gd0_tensor, torch.cat(target)), raise_exception=True)

test_gradcheck_attach()

True

In [9]:
def controlslist2tensor(controls_list):
    return torch.cat([controls_list[0].view(-1),controls_list[1].view(-1), controls_list[-1][0].view(-1),controls_list[-1][1].view(-1)])

def controlstensor2list(controls, nb_pts, dim):
    n = sum(nb_pts)
    return [controls[:dim*nb_pts[0]], controls[dim*nb_pts[0]:dim*(nb_pts[0]+nb_pts[1])], [controls[dim*n:dim*(n+nb_pts[0])], controls[dim*(n+nb_pts[0]):]]]

def test_gradcheck_cost():
    def cost(gd, cont):
        cont = controlstensor2list(cont, nb_pts, dim)
        gd = EnergyFunctional.tensor2list(gd)
        EnergyFunctional.modules.manifold.fill_gd(gd)
        EnergyFunctional.modules.fill_controls(cont)
        return EnergyFunctional.cost()
    
    return torch.autograd.gradcheck(cost, (gd0_tensor, controlslist2tensor(modules.controls)), raise_exception=True)

test_gradcheck_cost()

True

In [10]:
def test_gradcheck_shoot():
    def shoot(x, y):
        x = EnergyFunctional.tensor2list(x)
        y = EnergyFunctional.tensor2list(y)
        
        EnergyFunctional.modules.manifold.fill_gd(x)
        EnergyFunctional.modules.manifold.fill_cotan(y)  
        
        intermediate_states, intermediate_controls = EnergyFunctional.shoot()
        return EnergyFunctional.modules.manifold.gd[0]
    
    return torch.autograd.gradcheck(shoot, (gd0_tensor, mom0_tensor), raise_exception=True)

test_gradcheck_shoot()

True

In [11]:
def test_gradcheck_energy_tensor():
    def energy_tensor(x,y):
         
        return EnergyFunctional.energy_tensor(x,y)
    
    return torch.autograd.gradcheck(energy_tensor, (gd0_tensor, mom0_tensor), raise_exception=True)

test_gradcheck_energy_tensor()

True